In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

1. **Выбор числа *k***  
   - *k* — это количество ближайших соседей, которые будут учитываться при принятии решения.  
   - Например, *k=3* означает, что алгоритм рассмотрит 3 ближайших объекта.

2. **Вычисление расстояний**  
   - Для нового объекта вычисляются расстояния до всех точек в обучающей выборке.  
   - Популярные метрики расстояния:  
     - **Евклидово расстояние** (для непрерывных признаков)
     - **Манхэттенское расстояние** (для данных с выбросами)
     - **Косинусное расстояние** (для текстовых/векторных данных).  

3. **Выбор *k* ближайших соседей**  
   - Из всех объектов выбираются *k* с наименьшими расстояниями.

4. **Принятие решения**  
   - **Классификация**: выбирается класс, который чаще всего встречается среди *k* соседей (голосование).  
   - **Регрессия**: вычисляется среднее (или медиана) значений целевой переменной у соседей.

In [7]:
# Загрузка данных (датасет вина)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = pd.read_csv(url, sep=';')

In [25]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [22]:
# Быстрый анализ данных
data_info = data.info()
data_desc = data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [9]:
# Масштабирование данных (кроме колонки 'quality')
scaler = StandardScaler()
X = data.drop('quality', axis=1)
y = data['quality']

X_scaled = scaler.fit_transform(X)

In [10]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [11]:
# Обучение модели с K = 5
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

# Прогноз на тестовой выборке
y_pred = knn.predict(X_test)

In [12]:
# Расчет метрик
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R^2 Score: {r2}")

Mean Absolute Error (MAE): 0.505625
Mean Squared Error (MSE): 0.43862500000000004
R^2 Score: 0.3288124448960684


In [13]:
# Определяем параметры для перебора
param_grid = {'n_neighbors': list(range(1, 31))}

# Настраиваем кросс-валидацию
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Инициализация GridSearchCV
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=kf, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Лучшие параметры и метрика
best_k_grid = grid_search.best_params_
best_score_grid = -grid_search.best_score_

print(f"Лучшее значение K (GridSearchCV, KFold): {best_k_grid}")
print(f"Лучший MSE (GridSearchCV, KFold): {best_score_grid}")

Лучшее значение K (GridSearchCV, KFold): {'n_neighbors': 27}
Лучший MSE (GridSearchCV, KFold): 0.4487159586056645


In [14]:
# Настройка RandomizedSearchCV
random_search = RandomizedSearchCV(KNeighborsRegressor(), param_grid, cv=kf,
                                   scoring='neg_mean_squared_error', n_iter=10, random_state=42)
random_search.fit(X_train, y_train)

# Лучшие параметры и метрика
best_k_random = random_search.best_params_
best_score_random = -random_search.best_score_

print(f"Лучшее значение K (RandomizedSearchCV, KFold): {best_k_random}")
print(f"Лучший MSE (RandomizedSearchCV, KFold): {best_score_random}")

Лучшее значение K (RandomizedSearchCV, KFold): {'n_neighbors': 25}
Лучший MSE (RandomizedSearchCV, KFold): 0.4492688333333333


In [15]:
# Настройка стратифицированной кросс-валидации
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Инициализация GridSearchCV
grid_search_stratified = GridSearchCV(KNeighborsRegressor(), param_grid, cv=skf, scoring='neg_mean_squared_error')
grid_search_stratified.fit(X_train, y_train)

# Лучшие параметры и метрика
best_k_grid_stratified = grid_search_stratified.best_params_
best_score_grid_stratified = -grid_search_stratified.best_score_

print(f"Лучшее значение K (GridSearchCV, StratifiedKFold): {best_k_grid_stratified}")
print(f"Лучший MSE (GridSearchCV, StratifiedKFold): {best_score_grid_stratified}")


Лучшее значение K (GridSearchCV, StratifiedKFold): {'n_neighbors': 19}
Лучший MSE (GridSearchCV, StratifiedKFold): 0.4415687802129161


In [16]:
# Настройка RandomizedSearchCV с StratifiedKFold
random_search_stratified = RandomizedSearchCV(KNeighborsRegressor(), param_grid, cv=skf,
                                              scoring='neg_mean_squared_error', n_iter=10, random_state=42)
random_search_stratified.fit(X_train, y_train)

# Лучшие параметры и метрика
best_k_random_stratified = random_search_stratified.best_params_
best_score_random_stratified = -random_search_stratified.best_score_

print(f"Лучшее значение K (RandomizedSearchCV, StratifiedKFold): {best_k_random_stratified}")
print(f"Лучший MSE (RandomizedSearchCV, StratifiedKFold): {best_score_random_stratified}")


Лучшее значение K (RandomizedSearchCV, StratifiedKFold): {'n_neighbors': 24}
Лучший MSE (RandomizedSearchCV, StratifiedKFold): 0.4435867661951933


In [17]:
# Список лучших параметров
best_params = [
    best_k_grid['n_neighbors'],
    best_k_random['n_neighbors'],
    best_k_grid_stratified['n_neighbors'],
    best_k_random_stratified['n_neighbors']
]

# Имена моделей
model_names = [
    "GridSearchCV (KFold)",
    "RandomizedSearchCV (KFold)",
    "GridSearchCV (StratifiedKFold)",
    "RandomizedSearchCV (StratifiedKFold)"
]

# Словарь для хранения результатов
results = {}

# Оценка каждой модели
for name, k in zip(model_names, best_params):
    print(f"\nМодель: {name} с K = {k}")
    model = KNeighborsRegressor(n_neighbors=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Метрики
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Сохранение результатов
    results[name] = {'MAE': mae, 'MSE': mse, 'R^2': r2}

    # Вывод результатов
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"R^2 Score: {r2}")



Модель: GridSearchCV (KFold) с K = 27
Mean Absolute Error (MAE): 0.524537037037037
Mean Squared Error (MSE): 0.41243998628257883
R^2 Score: 0.36888096661133507

Модель: RandomizedSearchCV (KFold) с K = 25
Mean Absolute Error (MAE): 0.5261250000000001
Mean Squared Error (MSE): 0.413785
R^2 Score: 0.36682281564279207

Модель: GridSearchCV (StratifiedKFold) с K = 19
Mean Absolute Error (MAE): 0.5169407894736842
Mean Squared Error (MSE): 0.4082323407202216
R^2 Score: 0.3753195401941051

Модель: RandomizedSearchCV (StratifiedKFold) с K = 24
Mean Absolute Error (MAE): 0.523046875
Mean Squared Error (MSE): 0.4123426649305556
R^2 Score: 0.36902988853854823
